In [55]:
import warnings as wn
wn.filterwarnings('ignore')

In [56]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
df_train = pd.read_csv('./train.csv')
df_test = pd.read_csv('./test.csv')

In [57]:
df_train[df_train['Embarked'].isnull()]
df_train.loc[df_train['PassengerId'].isin([62, 830]), 'Embarked'] = 'C'

In [58]:
df_test.loc[df_test['PassengerId']== 1044, 'Fare']=13.675550

In [59]:
def impute_age(cols):
    Age = cols[0]
    Pclass = cols[1]
    if pd.isnull(Age):
        if Pclass == 1:
            return 39
        elif Pclass == 2:
            return 30
        else:
            return 25
    else:
        return Age
    
def impute_sex(cols):
    Sex = cols[0]
    if pd.isnull(Sex):
        if Sex == 'male':
            return 0
        else:
            return 1
    else:
        return 0
    
def impute_embarked(cols):
    Embarked = cols[0]
    if Embarked == 'S':
        return 0
    elif Embarked == 'C':
        return 1
    elif Embarked == 'Q':
        return 2
    else:
        return Embarked

data = [df_train, df_test]

for df in data:
    df['Age'] = df[['Age','Pclass']].apply(impute_age, axis = 1)
    
    df['Sex'] = df[['Sex']].apply(impute_sex, axis = 1)
    df['Embarked'] = df[['Embarked']].apply(impute_embarked, axis = 1)
    df.loc[df['Fare']<=7.91, 'Fare'] = 0
    df.loc[(df['Fare']>7.91) & (df['Fare']<=14.454), 'Fare'] = 1
    df.loc[(df['Fare']>14.454) & (df['Fare']<=31), 'Fare'] = 2
    df.loc[(df['Fare']>31), 'Fare'] = 3
    df['Fare'] = df['Fare'].astype(int)
    
    df.loc[df['Age']<=16, 'Age'] = 0
    df.loc[(df['Age']>16) & (df['Age']<=32), 'Age'] = 1
    df.loc[(df['Age']>32) & (df['Age']<=48), 'Age'] = 2
    df.loc[df['Age']>48, 'Age'] = 3
    df['Age'] = df['Age'].astype(int)
    
    df['FamilySize'] = df['SibSp'] + df['Parch'] + 1
    df['IsAlone'] = 0
    df.loc[df['FamilySize'] == 1, 'IsAlone'] = 1
    
df_train.drop(['Name', 'Cabin', 'Ticket', 'SibSp', 'Parch'], axis = 1, inplace = True)
df_test.drop(['Name', 'Cabin', 'Ticket', 'SibSp', 'Parch'], axis = 1, inplace = True)
    
#trainingデータの結果のみ抜き出し
#X_train = df_train.drop(['PassengerId','Survived'],axis= 1)
X_train = df_train.drop(['Survived'],axis= 1)

#trainingデータの結果のみ抜き出し
Y_train = df_train['Survived']
X_test = df_test.copy()
    
# 学習
forest = RandomForestClassifier(random_state=1)
forest.fit(X_train, Y_train)
Y_pred = forest.predict(X_test)

# 提出データの作成
submission = pd.DataFrame({
        "PassengerId": df_test["PassengerId"],
        "Survived": Y_pred})
submission.to_csv('submit3.csv', index=False)

In [60]:
for i,k in zip(X_train.columns,forest.feature_importances_):
  print(i,round(k,4))

PassengerId 0.5821
Pclass 0.095
Sex 0.0
Age 0.09
Fare 0.0833
Embarked 0.0506
FamilySize 0.0858
IsAlone 0.0132
